In [69]:
#Parameters

MaxPctAllocation = 1 # Maximum Allocation weight of 1 asset
MinPctAllocation = -1 # Minimum Allocation weight of 1 asset, -1 denotes able to short 
folder_name = "PortOpt"

import pandas as pd #No PANDA NO LIFE
import matplotlib.pyplot as plt #Plotting
import os #CSV File Reading
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

#For OS locating CSV folder
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, folder_name)

# List of CSV files
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]


def read_each_file(csv_files):
    # Initialize a dictionary to store each DataFrame
    dfs = {}

    # Loop over the list of CSV files
    for csv_file in csv_files:
        # Read the 'Exchange Date' and 'Close' columns from the current CSV file
        df = pd.read_csv(csv_file, usecols=['Exchange Date', 'Close'])

        # Converts 'Exchange Date' column into datetime format
        df['Exchange Date'] = pd.to_datetime(df['Exchange Date'])

        # Get the base name of the file (without extension)
        base_name = csv_file.split('.')[0]
        # Extract the last 3 characters from the base_name
        new_column_name = base_name[-3:]
        
        # Rename the 'Close' and 'Daily Returns' columns
        df.rename(columns={'Close': new_column_name}, inplace=True)

        # Store the DataFrame in the dictionary, using the file name as the key
        dfs[base_name] = df

    return dfs



# Read each file & conv into a df
dataframes = read_each_file(csv_files)

def merge_dataframes(dfs):
    # Convert the dictionary values to a list
    dfs_list = list(dfs.values())

    # Use the first DataFrame as the initial merged DataFrame
    merged_df = dfs_list[0]

    # Loop over the rest of the DataFrames
    for df in dfs_list[1:]:
        # Merge the current DataFrame into the merged DataFrame
        merged_df = pd.merge(merged_df, df, on='Exchange Date', how='inner')

    return merged_df

# merge function
portfolio  = merge_dataframes(dataframes)
#print(portfolio )


# Drop the first row, which will contain NaN values after calculating daily returns
portfolio = portfolio.dropna()

#print(portfolio)

# Calculate the expected returns
returns_df = portfolio.drop('Exchange Date', axis=1)
mu = mean_historical_return(returns_df)

#Cal S
S = CovarianceShrinkage(returns_df).ledoit_wolf()
ef = EfficientFrontier(mu, S,weight_bounds=(MinPctAllocation, MaxPctAllocation))

#Find Max Sharpe on curve
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))
ef.portfolio_performance(verbose=True)


{'GIM': 0.60932, 'HAS': 0.11736, 'YLD': 1.0, 'SPY': -1.0, 'TLT': 0.08764, 'USO': 0.18568}
Expected annual return: 23.8%
Annual volatility: 19.1%
Sharpe Ratio: 1.14


(0.23779155761148088, 0.19059333236818937, 1.1427029209539703)